In [1]:
# imports
import numpy as np
import pandas as pd
import random
import itertools
from sklearn.linear_model import LinearRegression

In [2]:
def generate_task_index(X_without_task_labels):
    # when we pass in X for the q2 calculation, we do not have the task labels
    # so, let's add in an index for each group of rows (which are part of the same task)

    num_rows = len(X_without_task_labels)
    num_rows_in_group = num_rows / 10 # because there are 10 tasks
    num_groups = (num_rows + num_rows_in_group - 1) // num_rows_in_group  # Calculate the number of groups, rounding up
    key_values = np.repeat(np.arange(num_groups), num_rows_in_group)[:num_rows]  # Generate key values for each group
    df = X_without_task_labels.copy()
    df['task_index'] = key_values

    return df

In [3]:
def reshape_x_y(x, y):
    if(isinstance(x, pd.Series)):
        x = np.asarray(x).reshape(-1, 1)
    else:
        x = np.asarray(x)
    
    y = np.asarray(y).reshape(-1, 1)
    return(x, y)

In [4]:
def q2_task_holdout_helper(x_train, x_test, y_train, y_test, estimator):

    # some reshaping
    x_train_array, y_train_array = reshape_x_y(x_train, y_train)
    x_test_array, y_test_array = reshape_x_y(x_test, y_test)

    # Fit the model and get the error
    fitted_model = estimator.fit(X=x_train_array, y=y_train_array.ravel())
    
    # save prediction error
    prediction = fitted_model.predict(x_test_array)[0]
    squared_model_prediction_error = (y_test_array - prediction) ** 2

    # save total error for this fold
    squared_average_prediction_error = (y_test_array - np.mean(y_train_array)) ** 2

    return squared_model_prediction_error, squared_average_prediction_error

In [5]:
"""
This is the version of q^2 that holds out EVERYTHING associated with a given task

It trains on all task instances from the "seen" classes, and it tests on task instances of held-out (unseen) classes.
"""

def get_q2(y, x, estimator = LinearRegression(), num_task_holdouts = 1, N_TASK_SAMPLES = 100):
    squared_model_prediction_errors = []
    squared_average_prediction_errors = []

    # generate an index for x and y
    x = generate_task_index(pd.DataFrame(x))
    y = generate_task_index(pd.DataFrame(y))

    num_total_tasks = len(set(x['task_index'].values))

    for i in range(num_total_tasks):        
        # randomly hold out `num_task_holdouts`
        all_possible_task_combos = list(itertools.combinations(set(x['task_index'].values), num_total_tasks - num_task_holdouts))
        task_samples = all_possible_task_combos# random.choices(all_possible_task_combos, k=N_TASK_SAMPLES)
        
        for sample in task_samples:
            x_train_tasks = x[x["task_index"].isin(sample)].drop('task_index', axis = 1)
            x_test_tasks = x[~x["task_index"].isin(sample)].drop('task_index', axis = 1)

            y_train_tasks = y[y["task_index"].isin(sample)].drop('task_index', axis = 1)
            y_test_tasks = y[~y["task_index"].isin(sample)].drop('task_index', axis = 1)

            # get evaluation score by training on the training tasks and evaluating on the holdout tasks
            squared_model_prediction_error, squared_average_prediction_error = q2_task_holdout_helper(x_train_tasks, x_test_tasks, y_train_tasks, y_test_tasks, estimator)

            squared_model_prediction_errors.append(squared_model_prediction_error)
            squared_average_prediction_errors.append(squared_average_prediction_error)

    return 1 - (np.sum(squared_model_prediction_errors) / np.sum(squared_average_prediction_errors))

# Linear Models for Strong Synergy

In [6]:
task_cols_to_use = ['Q1concept_behav', 'Q3type_1_planning', 'Q4type_2_generate',
       'Q6type_5_cc', 'Q7type_7_battle', 'Q8type_8_performance',
       'Q9divisible_unitary', 'Q10maximizing', 'Q11optimizing',
       'Q13outcome_multip', 'Q14sol_scheme_mul', 'Q15dec_verifiability',
       'Q16shared_knowledge', 'Q17within_sys_sol', 'Q18ans_recog',
       'Q19time_solvability', 'Q20type_3_type_4', 'Q22confl_tradeoffs',
       'Q23ss_out_uncert', 'Q24eureka_question', 'Q2intel_manip_1',
       'Q21intellective_judg_1', 'Q5creativity_input_1']

In [7]:
grouped_performance_with_task_cols = pd.read_csv('grouped_performance_with_task_cols.csv')

In [8]:
# test LM on the bootstrapped data
get_q2(grouped_performance_with_task_cols["strong_synergy"], grouped_performance_with_task_cols[list(task_cols_to_use) + ["playerCount", "Low", "Medium", "High"]])

-3.0762575698996276e+24

In [9]:
# test LM on the permutation data
get_q2(grouped_performance_with_task_cols["strong"], grouped_performance_with_task_cols[list(task_cols_to_use) + ["playerCount",  "Low", "Medium", "High"]])

-2.1250760073296147e+24